<img src="./figs/IOAI-Logo.png" alt="IOAI Logo" width="200" height="auto">

[IOAI 2025 (Beijing, China), Individual Contest](https://ioai-official.org/china-2025)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IOAI-official/IOAI-2025/blob/main/Individual-Contest/Pixel/Pixel.ipynb)

# Pixel Efficiency

## 1. Problem Description

You are a student in wildlife biology, working on a groundbreaking research project at the Starr Park Research Center. Your team has deployed thousands of camera traps across remote wilderness areas to monitor endangered species populations. However, the satellite internet connections in these remote locations have extremely limited bandwidth. Your job is to write code that identifies the most critical pixels in each wildlife photograph so that only the essential visual information needs to be transmitted back to headquarters.

<img src="./figs/Pixel Fig 1.png" width="400">

## 2. Dataset

The dataset consists of a training set and a test set. Datasets are loaded using `load_from_disk`, and are in the format of `datasets`. Test set is not visible to the contestants.

In the dataset there are the following fields:

- `image`: the image are RGB full color images in PIL format, the size of each image is (224, 224)
- `name`: the animal species label
- `idx`: unique identifiers used to track the records.

1. **Training Set (`train_dataset` folder)**:
    - The training set is used for training your models/ doing experimentations on and can be accessed and downloaded directly during the competition.
    - There are 700 images in the training set.

2. **Test Set (`test_dataset` folder)**: 
    - These follow the same format as the training set but do not contain the `name` field.
    - There are 698 images in test set, which had been separated into 2 testing sets within the ratio of 3:7, i.e. 30% of the data would be used to calculate the Leaderboard A score, another 70% data would be used to calculate the Leaderboard B score.
    - The testing set is used to calculate the Leaderboard A score and the Leaderboard B score and is not directly accessible during the competition. Contestants can access the result on Leaderboard A , but cannot access the result on Leaderboard B. The final score would be counted using Leaderboard B only. The subsets for Leaderboard A and Leaderboard B are completely distinct.
    

## 3. Task
You are given a dataset of animal photographs and a CLIP model that can do a zero-shot classification of animal species. To conserve bandwidth, you need to retain at most **6.25%** of the pixels of each image, while keeping classification accuracy as high as possible.

More specifically, your task is to return **one rectangle mask** for each image, which contain a single rectangular area indicating the area to keep. Each mask is defined by two coordinate tuples: one for the top-left corner and one for the bottom-right corner of the rectangle. Below is a visualization of what the image would look like after applying a rectangular mask using the process from the baseline:

<img src="./figs/Pixel Fig 2.png" width="400">

**Coordinate Convention:**
- Top-left corner coordinates are **inclusive** (the pixel at this position is included in the mask) 
- Bottom-right corner coordinates are **exclusive** (the pixel at this position is NOT included in the mask)

For example, if you specify coordinates `((10, 20), (15, 25))`, the mask will cover pixels from row 10 to 14 (inclusive) and column 20 to 24 (inclusive), for a total area of 25 pixels.

As an illustration, if an image size is 3x3 and we wanted to keep only the top-right pixel using coordinates `((0, 2), (1, 3))`, the resulting binary mask would be:

```
[[0, 0, 1],
 [0, 0, 0],
 [0, 0, 0]]
```

Below is a summary of the requirements for your masks:

- Return one rectangle mask defined by coordinate tuples: `((top, left), (bottom, right))`
- Top-left corner coordinates are inclusive, bottom-right corner coordinates are exclusive
- The rectangle mask should cover at most *6.25%* of the original pixels (minimum 93.75% reduction of the original pixels)
- All images are of size (224, 224), so coordinate values should be within the range [0, 224]


Images would be masked using the mask you created, outside the masked rectangle, all pixels outside the masked rectangle will be replaced with RGB(0, 0, 0) (black) values. The masked image will be then passed through the CLIP model during evaluation, and your task is to keep the classification accuracy of the CLIP model on these masked images as high as possible. **An additional `other` class would be added into the classes for classification** to ensure that your masked image retains actual useful information for the researchers back at Starr Park Headquarters. So for example, if your image doesn't contain any animal information, the model will predict the `others` class instead of predicting a random animal and having a chance of getting it correct.

You need to work only with the provided CLIP model and dataset. As a reminder, CLIP generates representations for both text and image, and it can compute a similarity score between them. So if you have ten animal classes, CLIP can look at the provided image and decide which text (class) is closest to the image. 

To ensure that your solution would handle the traffic of images for the research center, your code should run in **UNDER 8 MINUTES for the 698 images in the test dataset**. It is recommended that you test your solution on the training set first, which contain 700 images, to understand how much time your solution takes (testing set would take slightly longer due to dataset loading).

## 4. Submission

Contestants need to submit a notebook file named `submission.ipynb`. The file should output a `.jsonl` file titled `submission.jsonl`, which contains all the generated masks for the dataset split. Each mask in the `submission.jsonl` file should be stored as a tuple of two coordinate tuples: `((top, left), (bottom, right))`, where the top-left corner is inclusive and the bottom-right corner is exclusive.

Contestants don't need to separate test sets into Leaderboard A and Leaderboard B, the evaluation machine will read `submission.jsonl` and automatically calculate the scores for Leaderboard A and Leaderboard B based on the prediction results and true labels. 

The submission files must strictly follow the above format and naming; otherwise, the system will not be able to read them correctly. 

## 5. Score

The evaluation metric will be **classification accuracy**, defined as the proportion of correctly predicted samples over the total number of evaluated samples.

Your score is the zero-shot classification accuracy of CLIP on the masked test images. **If a submitted mask for an image is invalid (wrong shape, more than 6.25% pixels retained, etc.), that image is counted as incorrect. A sample script is provided to compute the training split score.**


## 6. Baseline and Training Set

- Below you can find the baseline solution.
- The dataset is in `training_set` folder.
- The highest score by the Scientific Committee for this task is 0.83 in Leader Board B, this score is used for score unification.
- The baseline score by the Scientific Committee for this task is 0.19 in Leader Board B, this score is used for score unification.

In [ ]:
# Install required packages
!pip install numpy pillow tqdm torch transformers datasets accelerate matplotlib

In [ ]:

from datasets import load_dataset
print("Loading dataset...")
# Load dataset directly from HuggingFace without data_dir parameter
# The correct way to load a specific subset/path from a HuggingFace dataset

DATASET_NAME = "IOAI-official/IOAI-2025-Pixel-train"
SPLIT = 'train'
load_dataset(DATASET_NAME, data_dir="data", split=SPLIT) 

print(f"Dataset loaded successfully! Total samples: {len(dataset)}")

# Print first item to check available fields
print("\nFirst item keys:")
print(dataset[0].keys())

# Show basic dataset statistics without converting fields yet
print(f"\nDataset loaded successfully!")
print(f"Total samples: {len(dataset)}")

print(f"\nSample item structure:")
sample_item = dataset[0]
print(f"  Keys: {list(sample_item.keys())}")
if 'image' in sample_item:
    print(f"  Image type: {type(sample_item['image'])}")
    print(f"  Image size: {sample_item['image'].size}")
else:
    print("  No 'image' field found in dataset")
    
if 'idx' in sample_item:
    print(f"  Index: {sample_item['idx']}")
else:
    print("  No 'idx' field found in dataset")



In [ ]:

print("Loading dataset...")
# Load dataset directly from HuggingFace without data_dir parameter
# The correct way to load a specific subset/path from a HuggingFace dataset
dataset = load_dataset(DATASET_NAME, data_files="Individual-Contest/Pixel/training_set/train/data-00000-of-00001.arrow", split=SPLIT)

print(f"Dataset loaded successfully! Total samples: {len(dataset)}")

# Print first item to check available fields
print("\nFirst item keys:")
print(dataset[0].keys())

# Show basic dataset statistics without converting fields yet
print(f"\nDataset loaded successfully!")
print(f"Total samples: {len(dataset)}")

print(f"\nSample item structure:")
sample_item = dataset[0]
print(f"  Keys: {list(sample_item.keys())}")
if 'image' in sample_item:
    print(f"  Image type: {type(sample_item['image'])}")
    print(f"  Image size: {sample_item['image'].size}")
else:
    print("  No 'image' field found in dataset")
    
if 'idx' in sample_item:
    print(f"  Index: {sample_item['idx']}")
else:
    print("  No 'idx' field found in dataset")



In [ ]:
import random
import numpy as np
import torch

seed = 42

random.seed(seed)                  # Python built-in random
np.random.seed(seed)               # NumPy
torch.manual_seed(seed)            # PyTorch (CPU)
torch.cuda.manual_seed(seed)       # PyTorch (single GPU)
torch.cuda.manual_seed_all(seed)   # PyTorch (all GPUs)

# Ensures deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Dependencies and Config Variables

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from PIL import Image
from tqdm import tqdm
import glob
import json
import math
import torch
import matplotlib.pyplot as plt
from datasets import load_dataset, load_from_disk
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from tqdm.auto import tqdm 


# Dataset configuration
DATASET_PATH = "IOAI-official/IOAI-2025-Pixel-train"
SPLIT = "train"

# Model Configuration
MODEL_PATH = "openai/clip-vit-large-patch14"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BACKGROUND_CLASS = "other"


# Image and Masking Configuration
HEIGHT = 224
WIDTH = 224
RETAIN_RATIO = 0.0625  # Retain 6.25% of pixels
MEAN_COLOR = (0, 0, 0)  # RGB mean values for masked out areas


### Dataset loading

Let's first load the dataset in and see what's in it:


In [ ]:

print("Loading dataset...")
# Load dataset directly from HuggingFace without data_dir parameter
dataset = load_dataset(DATASET_PATH, split=SPLIT)

print(f"Dataset loaded successfully! Total samples: {len(dataset)}")

# Print first item to check available fields
print("\nFirst item keys:")
print(dataset[0].keys())

# Show basic dataset statistics without converting fields yet
print(f"\nDataset loaded successfully!")
print(f"Total samples: {len(dataset)}")

print(f"\nSample item structure:")
sample_item = dataset[0]
print(f"  Keys: {list(sample_item.keys())}")
print(f"  Image type: {type(sample_item['image'])}")
print(f"  Image size: {sample_item['image'].size}")
print(f"  Index: {sample_item['idx']}")



In [ ]:
# Visualize first 10 samples
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
axes = axes.flatten()

print("Visualizing first 10 samples...")

for i in range(10):
    sample = dataset[i]
    image = sample['image']
    label = sample['name']
    
    axes[i].imshow(image)
    axes[i].set_title(f"{label}\n", fontsize=12)
    axes[i].axis('off')

plt.tight_layout()
plt.suptitle('First 10 Samples from Dataset', fontsize=16, y=1.02)
plt.show()


### Model

Now let's load the model and see some predictions:

In [ ]:
print(f"Loading CLIP model and processor: {MODEL_PATH}...")
model = CLIPModel.from_pretrained(MODEL_PATH).to(DEVICE)
processor = CLIPProcessor.from_pretrained(MODEL_PATH)
print("Model and processor loaded successfully.")

In [ ]:
image = dataset[0]['image']
# Visualize the image with its true label
plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.title(f"Sample Image\nTrue Label: {dataset[0]['name']}", fontsize=14)
plt.axis('off')
plt.show()


labels = sorted(list(set(dataset['name']))) + [BACKGROUND_CLASS]
text_inputs = processor(text=labels, return_tensors="pt", padding=True).to(DEVICE)
image_processed = processor(images=image, return_tensors="pt").to(DEVICE)
pixel_values = image_processed['pixel_values']
outputs_full = model(pixel_values=pixel_values, **text_inputs)
logits_full = outputs_full.logits_per_image  # Shape: (1, num_styles)
predicted_index_full = logits_full.argmax(dim=-1).item()

print(f"Predicted label: {labels[predicted_index_full]}")


### Baseline: A trivial masking method

We will now be implementing a trivial masking solution, one that randomly masks out 90% of the pixels.

In [ ]:
def generate_center_crop_coordinates(image):
    """
    Generate coordinates for a center crop mask.
    
    Returns:
        tuple: ((top, left), (bottom, right)) coordinates for the crop
    """
    H, W = image.size
    total_px = H * W
    k = int(total_px * RETAIN_RATIO)
    
    # Calculate side length of the square crop
    side_length = int(np.sqrt(k))
    
    # Calculate center coordinates
    center_h, center_w = H // 2, W // 2
    
    # Calculate crop boundaries
    half_side = side_length // 2
    top = max(0, center_h - half_side)
    left = max(0, center_w - half_side)
    bottom = min(H, top + side_length)
    right = min(W, left + side_length)
    
    return ((top, left), (bottom, right))

def generate_mask_from_coordinates(image, coordinates):
    """
    Generate a binary mask from crop coordinates.
    
    Parameters:
        image: PIL Image
        coordinates: tuple of ((top, left), (bottom, right))
    
    Returns:
        numpy array: Binary mask with 1s in the crop area
    """
    H, W = image.size
    mask = np.zeros((H, W), dtype=np.int8)
    
    (top, left), (bottom, right) = coordinates
    mask[top:bottom, left:right] = 1
    
    return mask

In [ ]:
def apply_mask_with_mean(image, mask, mean_rgb=MEAN_COLOR):
    """
    Apply arbitrary binary mask to image, replacing masked areas with mean values

    Parameters:
    - image: PIL Image (224x224)
    - mask: Binary numpy array or PIL Image (224x224) where 0 is the area to drop and 1 is the area to keep
    - mean_rgb: RGB mean values to use (default: from config)

    Returns: Modified PIL Image
    """
    # Convert images to numpy arrays
    img_array = np.array(image).copy()

    # Ensure mask is numpy array
    if isinstance(mask, Image.Image):
        mask_array = np.array(mask.convert('L')) > 127  # Convert to binary
    else:
        mask_array = mask > 0

    # Reshape mask for broadcasting with RGB
    mask_3d = np.stack([mask_array] * 3, axis=2)

    # Convert mean values to 0-255 range
    mean_values = np.array([int(m * 255) for m in mean_rgb])
    # Apply mask - replace areas where mask is 0 (drop) with mean values, keep areas where mask is 1
    img_array = np.where(mask_3d, img_array, mean_values.reshape(1, 1, 3))

    return Image.fromarray(img_array.astype(np.uint8))

In [ ]:
image = dataset[0]['image']
# Visualize the image with its true label
plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.title(f"Sample Image\nTrue Label: {dataset[0]['name']}", fontsize=14)
plt.axis('off')
plt.show()


labels = sorted(list(set(dataset['name']))) + [BACKGROUND_CLASS]
text_inputs = processor(text=labels, return_tensors="pt", padding=True).to(DEVICE)

mask = generate_mask_from_coordinates(image, generate_center_crop_coordinates(image))
image_masked = apply_mask_with_mean(image, mask)

plt.figure(figsize=(8, 6))
plt.imshow(image_masked)
plt.title(f"Masked Image\nTrue Label: {dataset[0]['name']}", fontsize=14)
plt.axis('off')
plt.show()

image_processed = processor(images=image_masked, return_tensors="pt").to(DEVICE)
pixel_values = image_processed['pixel_values']
outputs_full = model(pixel_values=pixel_values, **text_inputs)
logits_full = outputs_full.logits_per_image  # Shape: (1, num_styles)
predicted_index_full = logits_full.argmax(dim=-1).item()

print(f"Predicted label: {labels[predicted_index_full]}")

### Exporting the masks


In [ ]:
dataset = load_dataset("IOAI-official/IOAI-2025-Pixel-test", split="test")

In [ ]:
## Exporting results and validating on full dataset
RETAIN_RATIO = 0.0625

masks = {}
for item in tqdm(dataset):
    image = item['image']

    ## you should replace mask generation with your function
    coordinates = generate_center_crop_coordinates(image)
    
    # don't need to change below, it's just saving to file
    idx = item['idx']
    # For validation, we still need to generate the full mask
    mask = generate_mask_from_coordinates(image, coordinates)
    assert mask.shape == (224, 224), "Mask should be 224x224"
    assert mask.sum() <= RETAIN_RATIO * 224 * 224, "You should leave only 6.25% of pixels"
    
    # Save only the coordinates (topleft, bottomright) instead of the full mask
    masks[idx] = coordinates

# Save as JSONL (one JSON object per line) - much safer than pickle
with open('submission.jsonl', 'w') as f:
    for idx, coordinates in masks.items():
        json.dump({"idx": idx, "coordinates": coordinates}, f)
        f.write('\n')

print("Masks saved to submission.jsonl")

### Validation

In [ ]:
# # Validation code for generated masks

# def check_validity(coordinates):
#     """
#     Check if coordinates are valid according to the requirements.
#     Returns True if valid, False otherwise.
#     """
#     try:
#         # Check if coordinates is a tuple of two tuples
#         if not isinstance(coordinates, tuple) or len(coordinates) != 2:
#             print(f"Coordinates is not a tuple of two tuples")
#             return False
        
#         (top, left), (bottom, right) = coordinates
        
#         # Check if all coordinates are integers
#         if not all(isinstance(coord, (int, np.integer)) for coord in [top, left, bottom, right]):
#             print(f"Coordinates are not integers")
#             return False
        
#         # Check if coordinates are within image bounds
#         # For slicing mask[top:bottom, left:right], valid ranges are:
#         # top, left: [0, 223] (inclusive)
#         # bottom, right: [1, 224] (inclusive) since we need top < bottom and left < right
#         if not (0 <= top < 224 and 0 <= left < 224 and 1 <= bottom <= 224 and 1 <= right <= 224):
#             print(f"Coordinates are not within image bounds")
#             return False
        
#         # Check if top-left is actually top-left of bottom-right (proper ordering)
#         if not (top < bottom and left < right):
#             print(f"Top-left is not actually top-left of bottom-right")
#             return False
        
#         # Check that the crop area doesn't exceed RETAIN_RATIO
#         crop_area = (bottom - top) * (right - left)
#         max_area = RETAIN_RATIO * 224 * 224
#         if crop_area > max_area:
#             print(f"Crop area {crop_area} exceeds max area {max_area}")
#             return False
        
#         return True
#     except Exception:
#         return False




# def validate_masks(masks):
#     """Simple validation of generated masks on the dataset"""
#     correct = 0
#     total = 0

#     labels = sorted(list(set(dataset['name']))) + ['other']
#     text_inputs = processor(text=labels, return_tensors="pt", padding=True).to(DEVICE)

#     with torch.no_grad():
#         for item in tqdm(dataset, desc="Validating masks"):
#             idx = item['idx']
#             if idx not in masks:
#                 continue

#             if not check_validity(masks[idx]):
#                 continue
                
#             mask_coordinates = masks[idx]
#             image = item['image']
#             true_label = item['name']
            
#             # Apply mask to image
#             if image.mode != "RGB":
#                 image = image.convert("RGB")
            
#             mask = generate_mask_from_coordinates(image, mask_coordinates)

#             # Apply mask with mean color replacement
#             img_array = np.array(image).copy()
#             mask_array = mask > 0
#             mask_3d = np.stack([mask_array] * 3, axis=2)
#             mean_values = np.array([0, 0, 0])  # Black mean color
#             img_array = np.where(mask_3d, img_array, mean_values.reshape(1, 1, 3))
#             masked_image = Image.fromarray(img_array.astype(np.uint8))
            
#             # Get prediction on masked image
#             image_processed = processor(images=masked_image, return_tensors="pt").to(DEVICE)
#             pixel_values = image_processed['pixel_values']
#             outputs = model(pixel_values=pixel_values, **text_inputs)
#             logits = outputs.logits_per_image
#             predicted_idx = logits.argmax(dim=-1).item()
#             predicted_label = labels[predicted_idx]
            
#             # Check if prediction is correct
#             if predicted_label == true_label:
#                 correct += 1
#             total += 1
    
#     accuracy = correct / total if total > 0 else 0
#     print(f"Validation Results:")
#     print(f"Total samples: {total}")
#     print(f"Correct predictions: {correct}")
#     print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
#     return accuracy

# # Run validation
# accuracy = validate_masks(masks)
